In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoost, CatBoostClassifier

In [61]:
train = pd.read_csv('../data/train_without_nan.csv')

In [62]:
test = pd.read_csv('../data/test_without_nan.csv')

## Проверка что в данных нет пропущенных значений:

In [63]:
train.isna().sum().sum()

0

In [64]:
test.isna().sum().sum()

0

## Таргет для обучения модели

In [65]:
y = train['end_cluster_x']
y

0         {other}
1         {other}
2             {α}
3          {α, γ}
4             {α}
           ...   
199995    {other}
199996     {α, γ}
199997    {other}
199998         {}
199999         {}
Name: end_cluster_x, Length: 200000, dtype: object

## Загрузка списка наиболее информативных признаков:

In [66]:
f = pd.read_csv('feat_l.csv')
cols = f['Feature Id'].tolist()
cols

['start_cluster_x',
 'okved_x',
 'start_cluster',
 'start_cluster_y',
 'sum_deb_e_oper_3m_x',
 'balance_amt_min_x',
 'sum_cred_e_oper_1m_x',
 'balance_amt_max_x',
 'channel_code_x',
 'segment_x',
 'sum_cred_e_oper_3m_x',
 'city',
 'sum_deb_h_oper_3m_x',
 'cnt_cred_e_oper_3m_x',
 'index_city_code_x',
 'min_founderpres',
 'balance_amt_day_avg_x',
 'b_amt_min_trend',
 'sum_deb_e_oper_1m_x',
 'cnt_days_deb_h_oper_1m_x',
 'sum_of_paym_1y_x',
 'balance_amt_max_y',
 'balance_amt_min',
 'index_city_code',
 'cnt_days_cred_e_oper_1m_x',
 'cnt_deb_h_oper_1m_x',
 'sum_cred_e_oper_3m_y',
 'ft_registration_date_x',
 'index_city_code_y',
 'min_founderpres_x',
 'sum_cred_h_oper_3m',
 'cnt_b_oper_1m_x',
 'cnt_cred_e_oper_1m_x',
 'sum_of_paym_1y',
 'sum_deb_e_oper_3m',
 'balance_amt_day_avg_y',
 'balance_amt_min_y',
 'cnt_days_deb_e_oper_1m_x',
 'balance_amt_avg_y',
 'sum_deb_e_oper_3m_trend_long',
 'balance_amt_avg',
 'cnt_days_deb_e_oper_3m',
 'cnt_a_oper_1m_x',
 'sm_cred_e_op_trend_3m',
 'okved_y',
 

In [67]:
train = train[cols]
test = test[cols]

In [68]:
train.head(5)

,start_cluster_x,okved_x,start_cluster,start_cluster_y,sum_deb_e_oper_3m_x,balance_amt_min_x,sum_cred_e_oper_1m_x,balance_amt_max_x,channel_code_x,segment_x,...,b_amt_avg_trend,sum_of_paym_2m_x,ogrn_exist_months_y,ogrn_month_y,mx_founder_trend,cnt_cred_h_oper_3m_y,ft_registration_date_y,segment_y,sum_deb_e_oper_3m_y,ogrn_exist_months
0,"{α, γ}",okved_30,"{α, γ}","{α, γ}",0.346818,0.430042,0.148894,0.740253,channel_code_5,segment_1,...,-0.136804,0.403604,2.455375,ogrn_month_0,0.018291,0.298873,2.469917,segment_1,0.302226,2.437979
1,{other},okved_5,{other},{other},0.028051,-0.114119,-0.008843,-0.114275,channel_code_2,segment_1,...,-0.001559,-0.092059,0.193904,ogrn_month_5,0.018550,0.255545,0.191770,segment_1,0.068519,0.176508
2,{α},okved_10,{α},{α},-0.144394,-0.112416,-0.132435,-0.187003,channel_code_12,segment_3,...,0.003342,-0.255946,0.298279,ogrn_month_9,0.018092,0.252657,0.311914,segment_3,-0.144523,0.280883
3,{α},okved_10,{α},{α},-0.152954,-0.125759,-0.143862,-0.204718,channel_code_14,segment_3,...,0.000089,-0.273969,-0.414954,ogrn_month_4,0.018291,0.250924,-0.407230,segment_3,-0.152925,-0.432350
4,{α},okved_2,{α},{α},-0.086248,-0.125672,-0.099270,-0.170262,channel_code_8,segment_2,...,-0.005188,-0.166740,0.420051,ogrn_month_4,0.018596,0.253234,0.427480,segment_2,-0.070999,0.402655


In [69]:
test.head(5)

,start_cluster_x,okved_x,start_cluster,start_cluster_y,sum_deb_e_oper_3m_x,balance_amt_min_x,sum_cred_e_oper_1m_x,balance_amt_max_x,channel_code_x,segment_x,...,b_amt_avg_trend,sum_of_paym_2m_x,ogrn_exist_months_y,ogrn_month_y,mx_founder_trend,cnt_cred_h_oper_3m_y,ft_registration_date_y,segment_y,sum_deb_e_oper_3m_y,ogrn_exist_months
0,{α},okved_0,{α},{α},0.149694,-0.125995,0.090933,0.049418,channel_code_12,segment_3,...,0.071039,0.223961,-0.641101,ogrn_month_8,0.018092,0.264211,-0.636647,segment_3,0.195415,-0.658497
1,{α},okved_6,{α},{α},-0.152970,-0.125856,-0.143862,-0.204920,channel_code_9,segment_3,...,0.000000,-0.211625,-0.641101,ogrn_month_10,0.018092,0.252516,-0.645800,segment_3,-0.152970,-0.658497
2,{other},okved_63,{other},{other},1.804880,-0.125995,1.193774,0.448252,channel_code_12,segment_3,...,-0.191179,4.142519,-0.745477,ogrn_month_6,0.019342,0.256701,-0.739055,segment_3,1.642996,-0.762873
3,{α},okved_5,{α},{α},-0.152970,-0.125501,-0.143862,-0.204813,channel_code_7,segment_3,...,0.000015,-0.211625,-0.902040,ogrn_month_9,0.018092,0.252516,-0.891236,segment_3,-0.152970,-0.919436
4,{α},okved_22,{},{},-0.152852,-0.125995,-0.142214,-0.201932,channel_code_7,segment_3,...,0.000000,-0.273794,-0.211019,ogrn_month_3,0.011265,0.250970,-0.207251,segment_3,-0.152970,-0.171906


## Разделение данных на выборки для обучения и тестирования:

In [17]:
x_train, x_val, y_train, y_val = train_test_split(train, y,
                                                  test_size=0.2,
                                                  random_state=42)

## Создание списка категориальных переменных для Catboost

In [19]:
cat_f = []
for col in train.columns:
    if train[col].dtypes in ['category', 'object']:
        cat_f.append(col)

## Подбор гиперпараметров и кроссвалидация

In [205]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

model  = CatBoostClassifier(random_state= 42,
                                 cat_features = cat_f,
                                 task_type='GPU',
                                 loss_function='MultiClassOneVsAll',
                                 )

param_distributions = {
    'iterations': np.arange(50, 1300, 35),
    'learning_rate': np.linspace(0.005, 1, 100),
    'l2_leaf_reg': np.linspace(1, 10, 60),
    'depth': [3, 4, 6, 5, 7, 8, 10, 17, 2, 30],
    "min_data_in_leaf": np.arange(5, 1000, 10)
}
randomized_search_result = model.randomized_search(param_distributions,
                                                   stratified=True,
                                                   cv = 5,
                                                   n_iter = 12,
                                                   X=train,
                                                   train_size=0.8,
                                                   y=y,
                                                   plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.5138227	test: 0.5138756	best: 0.5138756 (0)	total: 21.3ms	remaining: 19.7s
1:	learn: 0.3996066	test: 0.3997009	best: 0.3997009 (1)	total: 41.1ms	remaining: 19s
2:	learn: 0.3208253	test: 0.3209256	best: 0.3209256 (2)	total: 60.2ms	remaining: 18.5s
3:	learn: 0.2642420	test: 0.2643973	best: 0.2643973 (3)	total: 81ms	remaining: 18.6s
4:	learn: 0.2224837	test: 0.2226790	best: 0.2226790 (4)	total: 101ms	remaining: 18.6s
5:	learn: 0.1910910	test: 0.1912706	best: 0.1912706 (5)	total: 121ms	remaining: 18.5s
6:	learn: 0.1674423	test: 0.1676494	best: 0.1676494 (6)	total: 142ms	remaining: 18.7s
7:	learn: 0.1490201	test: 0.1492567	best: 0.1492567 (7)	total: 162ms	remaining: 18.6s
8:	learn: 0.1349127	test: 0.1351557	best: 0.1351557 (8)	total: 183ms	remaining: 18.6s
9:	learn: 0.1242136	test: 0.1245110	best: 0.1245110 (9)	total: 204ms	remaining: 18.6s
10:	learn: 0.1155703	test: 0.1158805	best: 0.1158805 (10)	total: 224ms	remaining: 18.6s
11:	learn: 0.1088598	test: 0.1091257	best: 0.1091257

95:	learn: 0.0793720	test: 0.0809307	best: 0.0809307 (95)	total: 1.92s	remaining: 16.6s
96:	learn: 0.0793494	test: 0.0809134	best: 0.0809134 (96)	total: 1.95s	remaining: 16.6s
97:	learn: 0.0793162	test: 0.0808924	best: 0.0808924 (97)	total: 1.97s	remaining: 16.6s
98:	learn: 0.0792900	test: 0.0808836	best: 0.0808836 (98)	total: 1.99s	remaining: 16.6s
99:	learn: 0.0792642	test: 0.0808674	best: 0.0808674 (99)	total: 2s	remaining: 16.5s
100:	learn: 0.0792392	test: 0.0808603	best: 0.0808603 (100)	total: 2.02s	remaining: 16.5s
101:	learn: 0.0791982	test: 0.0808298	best: 0.0808298 (101)	total: 2.04s	remaining: 16.5s
102:	learn: 0.0791784	test: 0.0808153	best: 0.0808153 (102)	total: 2.06s	remaining: 16.5s
103:	learn: 0.0791504	test: 0.0807991	best: 0.0807991 (103)	total: 2.08s	remaining: 16.4s
104:	learn: 0.0791264	test: 0.0807857	best: 0.0807857 (104)	total: 2.1s	remaining: 16.4s
105:	learn: 0.0791067	test: 0.0807765	best: 0.0807765 (105)	total: 2.12s	remaining: 16.4s
106:	learn: 0.0790822	te

187:	learn: 0.0775815	test: 0.0802297	best: 0.0802297 (187)	total: 3.65s	remaining: 14.3s
188:	learn: 0.0775712	test: 0.0802290	best: 0.0802290 (188)	total: 3.67s	remaining: 14.3s
189:	learn: 0.0775588	test: 0.0802291	best: 0.0802290 (188)	total: 3.69s	remaining: 14.3s
190:	learn: 0.0775374	test: 0.0802233	best: 0.0802233 (190)	total: 3.71s	remaining: 14.3s
191:	learn: 0.0775255	test: 0.0802235	best: 0.0802233 (190)	total: 3.73s	remaining: 14.2s
192:	learn: 0.0775078	test: 0.0802171	best: 0.0802171 (192)	total: 3.75s	remaining: 14.2s
193:	learn: 0.0774956	test: 0.0802158	best: 0.0802158 (193)	total: 3.77s	remaining: 14.2s
194:	learn: 0.0774795	test: 0.0802118	best: 0.0802118 (194)	total: 3.79s	remaining: 14.2s
195:	learn: 0.0774623	test: 0.0802054	best: 0.0802054 (195)	total: 3.81s	remaining: 14.2s
196:	learn: 0.0774461	test: 0.0801932	best: 0.0801932 (196)	total: 3.83s	remaining: 14.1s
197:	learn: 0.0774229	test: 0.0801818	best: 0.0801818 (197)	total: 3.85s	remaining: 14.1s
198:	learn

281:	learn: 0.0763783	test: 0.0799425	best: 0.0799425 (281)	total: 5.41s	remaining: 12.3s
282:	learn: 0.0763689	test: 0.0799462	best: 0.0799425 (281)	total: 5.43s	remaining: 12.3s
283:	learn: 0.0763497	test: 0.0799432	best: 0.0799425 (281)	total: 5.45s	remaining: 12.3s
284:	learn: 0.0763402	test: 0.0799448	best: 0.0799425 (281)	total: 5.46s	remaining: 12.3s
285:	learn: 0.0763225	test: 0.0799427	best: 0.0799425 (281)	total: 5.48s	remaining: 12.3s
286:	learn: 0.0763119	test: 0.0799417	best: 0.0799417 (286)	total: 5.5s	remaining: 12.2s
287:	learn: 0.0763005	test: 0.0799422	best: 0.0799417 (286)	total: 5.52s	remaining: 12.2s
288:	learn: 0.0762933	test: 0.0799420	best: 0.0799417 (286)	total: 5.54s	remaining: 12.2s
289:	learn: 0.0762875	test: 0.0799431	best: 0.0799417 (286)	total: 5.55s	remaining: 12.2s
290:	learn: 0.0762766	test: 0.0799456	best: 0.0799417 (286)	total: 5.57s	remaining: 12.1s
291:	learn: 0.0762557	test: 0.0799375	best: 0.0799375 (291)	total: 5.59s	remaining: 12.1s
292:	learn:

...

490:	learn: 0.0736135	test: 0.0800157	best: 0.0800138 (489)	total: 8.95s	remaining: 7.91s
491:	learn: 0.0736013	test: 0.0800125	best: 0.0800125 (491)	total: 8.96s	remaining: 7.89s
492:	learn: 0.0735947	test: 0.0800114	best: 0.0800114 (492)	total: 8.98s	remaining: 7.87s
493:	learn: 0.0735783	test: 0.0800089	best: 0.0800089 (493)	total: 9s	remaining: 7.85s
494:	learn: 0.0735686	test: 0.0800094	best: 0.0800089 (493)	total: 9.02s	remaining: 7.84s
495:	learn: 0.0735579	test: 0.0800074	best: 0.0800074 (495)	total: 9.04s	remaining: 7.82s
496:	learn: 0.0735466	test: 0.0800080	best: 0.0800074 (495)	total: 9.05s	remaining: 7.8s
497:	learn: 0.0735381	test: 0.0800066	best: 0.0800066 (497)	total: 9.07s	remaining: 7.78s
498:	learn: 0.0735276	test: 0.0800035	best: 0.0800035 (498)	total: 9.09s	remaining: 7.76s
499:	learn: 0.0735197	test: 0.0800077	best: 0.0800035 (498)	total: 9.11s	remaining: 7.75s
500:	learn: 0.0735023	test: 0.0800029	best: 0.0800029 (500)	total: 9.13s	remaining: 7.73s
501:	learn: 0.

583:	learn: 0.0726022	test: 0.0800177	best: 0.0799998 (541)	total: 10.7s	remaining: 6.25s
584:	learn: 0.0725999	test: 0.0800175	best: 0.0799998 (541)	total: 10.7s	remaining: 6.23s
585:	learn: 0.0725907	test: 0.0800175	best: 0.0799998 (541)	total: 10.7s	remaining: 6.21s
586:	learn: 0.0725797	test: 0.0800201	best: 0.0799998 (541)	total: 10.8s	remaining: 6.19s
587:	learn: 0.0725669	test: 0.0800217	best: 0.0799998 (541)	total: 10.8s	remaining: 6.17s
588:	learn: 0.0725559	test: 0.0800174	best: 0.0799998 (541)	total: 10.8s	remaining: 6.16s
589:	learn: 0.0725481	test: 0.0800187	best: 0.0799998 (541)	total: 10.8s	remaining: 6.14s
590:	learn: 0.0725383	test: 0.0800207	best: 0.0799998 (541)	total: 10.8s	remaining: 6.12s
591:	learn: 0.0725353	test: 0.0800214	best: 0.0799998 (541)	total: 10.8s	remaining: 6.1s
592:	learn: 0.0725282	test: 0.0800194	best: 0.0799998 (541)	total: 10.9s	remaining: 6.08s
593:	learn: 0.0725261	test: 0.0800196	best: 0.0799998 (541)	total: 10.9s	remaining: 6.06s
594:	learn:

679:	learn: 0.0716379	test: 0.0800396	best: 0.0799998 (541)	total: 12.4s	remaining: 4.48s
680:	learn: 0.0716279	test: 0.0800429	best: 0.0799998 (541)	total: 12.5s	remaining: 4.47s
681:	learn: 0.0716143	test: 0.0800434	best: 0.0799998 (541)	total: 12.5s	remaining: 4.45s
682:	learn: 0.0716011	test: 0.0800443	best: 0.0799998 (541)	total: 12.5s	remaining: 4.43s
683:	learn: 0.0715882	test: 0.0800406	best: 0.0799998 (541)	total: 12.5s	remaining: 4.41s
684:	learn: 0.0715816	test: 0.0800404	best: 0.0799998 (541)	total: 12.5s	remaining: 4.39s
685:	learn: 0.0715737	test: 0.0800387	best: 0.0799998 (541)	total: 12.6s	remaining: 4.37s
686:	learn: 0.0715650	test: 0.0800393	best: 0.0799998 (541)	total: 12.6s	remaining: 4.36s
687:	learn: 0.0715524	test: 0.0800431	best: 0.0799998 (541)	total: 12.6s	remaining: 4.34s
688:	learn: 0.0715395	test: 0.0800447	best: 0.0799998 (541)	total: 12.6s	remaining: 4.32s
689:	learn: 0.0715301	test: 0.0800449	best: 0.0799998 (541)	total: 12.6s	remaining: 4.3s
690:	learn:

772:	learn: 0.0707529	test: 0.0800704	best: 0.0799998 (541)	total: 14.2s	remaining: 2.79s
773:	learn: 0.0707419	test: 0.0800722	best: 0.0799998 (541)	total: 14.2s	remaining: 2.77s
774:	learn: 0.0707380	test: 0.0800745	best: 0.0799998 (541)	total: 14.2s	remaining: 2.75s
775:	learn: 0.0707251	test: 0.0800782	best: 0.0799998 (541)	total: 14.2s	remaining: 2.73s
776:	learn: 0.0707138	test: 0.0800764	best: 0.0799998 (541)	total: 14.3s	remaining: 2.72s
777:	learn: 0.0707021	test: 0.0800764	best: 0.0799998 (541)	total: 14.3s	remaining: 2.7s
778:	learn: 0.0706933	test: 0.0800767	best: 0.0799998 (541)	total: 14.3s	remaining: 2.68s
779:	learn: 0.0706816	test: 0.0800768	best: 0.0799998 (541)	total: 14.3s	remaining: 2.66s
780:	learn: 0.0706682	test: 0.0800778	best: 0.0799998 (541)	total: 14.3s	remaining: 2.64s
781:	learn: 0.0706581	test: 0.0800769	best: 0.0799998 (541)	total: 14.3s	remaining: 2.62s
782:	learn: 0.0706493	test: 0.0800782	best: 0.0799998 (541)	total: 14.4s	remaining: 2.6s
783:	learn: 

866:	learn: 0.0698789	test: 0.0801417	best: 0.0799998 (541)	total: 15.9s	remaining: 1.06s
867:	learn: 0.0698657	test: 0.0801402	best: 0.0799998 (541)	total: 15.9s	remaining: 1.05s
868:	learn: 0.0698599	test: 0.0801422	best: 0.0799998 (541)	total: 16s	remaining: 1.03s
869:	learn: 0.0698524	test: 0.0801443	best: 0.0799998 (541)	total: 16s	remaining: 1.01s
870:	learn: 0.0698428	test: 0.0801421	best: 0.0799998 (541)	total: 16s	remaining: 992ms
871:	learn: 0.0698348	test: 0.0801425	best: 0.0799998 (541)	total: 16s	remaining: 974ms
872:	learn: 0.0698214	test: 0.0801425	best: 0.0799998 (541)	total: 16s	remaining: 955ms
873:	learn: 0.0698151	test: 0.0801432	best: 0.0799998 (541)	total: 16.1s	remaining: 937ms
874:	learn: 0.0697951	test: 0.0801450	best: 0.0799998 (541)	total: 16.1s	remaining: 919ms
875:	learn: 0.0697893	test: 0.0801432	best: 0.0799998 (541)	total: 16.1s	remaining: 900ms
876:	learn: 0.0697791	test: 0.0801436	best: 0.0799998 (541)	total: 16.1s	remaining: 882ms
877:	learn: 0.069772

## Обучение модели с использованием оптимальных гиперпараметров:

### В процессе разработки, один из членов команды стер запись с выводом оптимальных гиперпараметров, но в модели ниже используются именно они

In [28]:
best_model  = CatBoostClassifier(random_state= 42,
                                 cat_features = cat_f,
                                 task_type='GPU',
                                 min_data_in_leaf=560,
                                 iterations=715, 
                                 learning_rate=0.173,
                                 l2_leaf_reg= 9.23728813559322,
                                 depth=8,
                                 loss_function='MultiClassOneVsAll',
                                 custom_metric=['AUC:hints=skip_train~false'])
best_model.fit(x_train,y_train, 
              eval_set=(x_val, y_val),
              plot=True,
              verbose=25
              )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.5534599	test: 0.5519061	best: 0.5519061 (0)	total: 186ms	remaining: 2m 13s
25:	learn: 0.0889235	test: 0.0819808	best: 0.0819808 (25)	total: 2.13s	remaining: 56.3s
50:	learn: 0.0822774	test: 0.0759628	best: 0.0759628 (50)	total: 4s	remaining: 52.2s
75:	learn: 0.0805795	test: 0.0749546	best: 0.0749546 (75)	total: 5.86s	remaining: 49.3s
100:	learn: 0.0794786	test: 0.0746130	best: 0.0746130 (100)	total: 7.68s	remaining: 46.7s
125:	learn: 0.0786572	test: 0.0743860	best: 0.0743860 (125)	total: 9.47s	remaining: 44.3s
150:	learn: 0.0780235	test: 0.0742417	best: 0.0742417 (150)	total: 11.4s	remaining: 42.5s
175:	learn: 0.0774650	test: 0.0741182	best: 0.0741182 (175)	total: 13.4s	remaining: 41.1s
200:	learn: 0.0769544	test: 0.0740385	best: 0.0740385 (200)	total: 15.4s	remaining: 39.5s
225:	learn: 0.0764934	test: 0.0739714	best: 0.0739714 (225)	total: 17.5s	remaining: 37.8s
250:	learn: 0.0760204	test: 0.0739334	best: 0.0739269 (245)	total: 19.7s	remaining: 36.5s
275:	learn: 0.0755225	

## Расчет целевой метрики над тестовой выборкой: 

In [31]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)
cluster_weights = pd.read_excel("cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [32]:
yp_proba = best_model.predict_proba(x_val)
y_pred_normalized = yp_proba / yp_proba.sum(axis=1, keepdims=True)
weighted_roc_auc(y_val, y_pred_normalized, best_model.classes_, weights_dict)

0.9128541539368511

## Значимость наиболее весомых признаков, используемых при обучении:

In [70]:
feat = best_model.get_feature_importance(prettified=True).head(30)
feat

,Feature Id,Importances
0,start_cluster_x,11.804954
1,city,3.406136
2,start_cluster,3.318654
3,okved_y,3.250499
4,index_city_code_x,3.046598
5,balance_amt_min,2.144290
6,balance_amt_min_x,2.124832
7,cnt_days_cred_e_oper_1m_x,1.913515
8,min_founderpres_x,1.740225
9,okved,1.710976


## Предсказания модели для тестируемой выборкой: 

In [41]:
test_pred_proba = best_model.predict_proba(test)
test_pred_proba = test_pred_proba / test_pred_proba.sum(axis=1, keepdims=True)
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=best_model.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

In [42]:
test_pred_proba_df

,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,0.010250,0.015397,0.016071,0.013615,0.004677,0.000653,0.002462,0.000630,0.007483,0.006620,0.025093,0.000407,0.002613,0.000077,0.003724,0.890182,0.000046
1,0.006492,0.394423,0.000520,0.002205,0.000763,0.000093,0.000842,0.000021,0.000797,0.003122,0.001290,0.000248,0.000643,0.000002,0.000906,0.587193,0.000439
2,0.580711,0.004134,0.013397,0.129051,0.006024,0.002891,0.005066,0.002549,0.066171,0.007327,0.011485,0.006983,0.003893,0.000181,0.015451,0.144581,0.000106
3,0.015444,0.741812,0.000494,0.000818,0.000355,0.000374,0.000467,0.000025,0.000619,0.015291,0.002901,0.000035,0.000532,0.000002,0.000238,0.220554,0.000041
4,0.088429,0.107816,0.005354,0.019406,0.003866,0.003404,0.000670,0.000074,0.004004,0.204029,0.004800,0.000351,0.016766,0.000003,0.000639,0.540253,0.000136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.015497,0.346630,0.001422,0.004876,0.000592,0.000052,0.000822,0.000017,0.001016,0.001043,0.001144,0.000228,0.000253,0.000008,0.000917,0.625403,0.000080
99996,0.022454,0.035299,0.014764,0.041733,0.006449,0.000746,0.001128,0.000077,0.006518,0.009525,0.005846,0.001091,0.003076,0.000008,0.001802,0.849276,0.000206
99997,0.022841,0.021344,0.040736,0.051693,0.018131,0.000136,0.001890,0.000215,0.012787,0.010422,0.004631,0.000846,0.003392,0.000040,0.018584,0.792288,0.000023
99998,0.052705,0.118308,0.024216,0.096958,0.006596,0.001167,0.000709,0.000122,0.012562,0.008826,0.010474,0.005882,0.001085,0.000005,0.004218,0.655757,0.000410


## Преобразоване датасета для тестирования на лидерборде

In [44]:
submission_df = pd.read_csv("sample_submission.csv")
submission_df[sorted_classes] = test_pred_proba_df
submission_df.to_csv("Submit.csv", index=False)

In [45]:
submission_df

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.010250,0.015397,0.016071,0.013615,0.004677,0.000653,0.002462,0.000630,0.007483,0.006620,0.025093,0.000407,0.002613,0.000077,0.003724,0.890182,0.000046
1,200001,0.006492,0.394423,0.000520,0.002205,0.000763,0.000093,0.000842,0.000021,0.000797,0.003122,0.001290,0.000248,0.000643,0.000002,0.000906,0.587193,0.000439
2,200002,0.580711,0.004134,0.013397,0.129051,0.006024,0.002891,0.005066,0.002549,0.066171,0.007327,0.011485,0.006983,0.003893,0.000181,0.015451,0.144581,0.000106
3,200003,0.015444,0.741812,0.000494,0.000818,0.000355,0.000374,0.000467,0.000025,0.000619,0.015291,0.002901,0.000035,0.000532,0.000002,0.000238,0.220554,0.000041
4,200004,0.088429,0.107816,0.005354,0.019406,0.003866,0.003404,0.000670,0.000074,0.004004,0.204029,0.004800,0.000351,0.016766,0.000003,0.000639,0.540253,0.000136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,299995,0.015497,0.346630,0.001422,0.004876,0.000592,0.000052,0.000822,0.000017,0.001016,0.001043,0.001144,0.000228,0.000253,0.000008,0.000917,0.625403,0.000080
99996,299996,0.022454,0.035299,0.014764,0.041733,0.006449,0.000746,0.001128,0.000077,0.006518,0.009525,0.005846,0.001091,0.003076,0.000008,0.001802,0.849276,0.000206
99997,299997,0.022841,0.021344,0.040736,0.051693,0.018131,0.000136,0.001890,0.000215,0.012787,0.010422,0.004631,0.000846,0.003392,0.000040,0.018584,0.792288,0.000023
99998,299998,0.052705,0.118308,0.024216,0.096958,0.006596,0.001167,0.000709,0.000122,0.012562,0.008826,0.010474,0.005882,0.001085,0.000005,0.004218,0.655757,0.000410
